### Basic Setup of Model
- Reads API key by default from the local env variable in the .env file `ANTHROPIC_API_KEY`
- Setup the API key by creating an account in Anthropic portal

In [ ]:
# Basic Setup of the client using Anthropic
from anthropic import Anthropic
# Load environment variables
from dotenv import load_dotenv
import os
import base64
import mimetypes
from IPython.display import Image
load_dotenv()
MODEL_NAME= "claude-3-5-sonnet-20241022"
client = Anthropic()

### Block of code to read user input and accept an image for analysis

In [ ]:
# Define a Image reader helper function
def create_image_message(image_path):
    print(image_path)
    with open(image_path, "rb") as file:
        # Read content of the image in binary format
        binary_data = file.read()
        print("binary data read !")
    # Convert binary data to base64 encoded string
    base64_data = base64.b64encode(binary_data).decode("utf-8")
    print("base64 data created !")
    # Get mimetype of the image
    mimetype, _ = mimetypes.guess_type(image_path)

    # Create a message object with the image data
    image_message = {
        "type": "image",
        "source": {
            "data": base64_data,
            "media_type": mimetype,
            "type": "base64",
        }
    }
    return image_message


user_input = "What can you tell about this image, it's origin and significance ?"
image_input = "image.jpg"
messages = [
    {
        "role": "user",
        "content": [
            create_image_message(image_input),
            {"type": "text", "text": user_input}
        ]
    }
]
print(messages)

#### Print response all at once
- This method will let the user wait longer to load the message until entire response is obtained

In [ ]:
# Print the whole message at once (longer wait for user)
response = client.messages.create(
    model=MODEL_NAME,
    max_tokens=1000,
    messages=messages,
)

print(response)

#### Stream the response 
- This approach prints the response as it comes through, thus reducing the wait time for user to see first response

In [ ]:
# Stream messages to the model (shorter wait for the user)
with client.messages.stream(
    model=MODEL_NAME,
    max_tokens=1000,
    messages=messages,
) as stream:
    for text in stream.text_stream:
        print(text, end="", flush=True)